Import libraries

In [1]:
from dataset.dataset import load_data
from models import MRnet
from config import config
import torch
from torch.utils.tensorboard import SummaryWriter
from utils.utils import _train_model, _evaluate_model, _get_lr
import time
import torch.utils.data as data
import os

Method for training a model

In [2]:
def train(config : dict):
    """
    Performs training of a specified model.

    Args:
        config (dict): Configuration to train with
    """
    
    print('Starting to Train Model...')

    # Load training, validation, and test data
    train_loader, val_loader, test_loader, train_wts, val_wts, test_wts = load_data()

    print('Initializing Model...')
    model = MRnet()
    if torch.cuda.is_available():
        model = model.cuda()
        train_wts = train_wts.cuda()
        val_wts = val_wts.cuda()

    print('Initializing Loss Method...')
    criterion = torch.nn.BCEWithLogitsLoss(pos_weight=train_wts)
    val_criterion = torch.nn.BCEWithLogitsLoss(pos_weight=val_wts)

    if torch.cuda.is_available():
        criterion = criterion.cuda()
        val_criterion = val_criterion.cuda()

    print('Setup the Optimizer')
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, patience=3, factor=.3, threshold=1e-4, verbose=True)
    
    starting_epoch = config['starting_epoch']
    num_epochs = config['max_epoch']
    patience = config['patience']
    log_train = config['log_train']
    log_val = config['log_val']

    best_val_loss = float('inf')
    best_val_acc = float(0)

    print('Starting Training')

    writer = SummaryWriter(comment='lr={} task={}'.format(config['lr'], config['task']))
    t_start_training = time.time()

    for epoch in range(starting_epoch, num_epochs):

        current_lr = _get_lr(optimizer)
        epoch_start_time = time.time()  # timer for entire epoch

        print('Started Training')
        train_loss, train_acc = _train_model(
            model, train_loader, epoch, num_epochs, optimizer, criterion, writer, current_lr, log_every=log_train)

        print('train loop ended, now val')
        val_loss, val_acc = _evaluate_model(
            model, val_loader, val_criterion,  epoch, num_epochs, writer, current_lr, log_val)

        writer.add_scalar('Train/Avg Loss', train_loss, epoch)
        writer.add_scalar('Val/Avg Loss', val_loss, epoch)

        scheduler.step(val_loss)

        t_end = time.time()
        delta = t_end - epoch_start_time

        print("train loss : {0} | train acc {1} | val loss {2} | val acc {3} | elapsed time {4} s".format(
            train_loss, train_acc, val_loss, val_acc, delta))

        print('-' * 30)

        writer.flush()

        if val_acc > best_val_acc:
            best_val_acc = val_acc

        if bool(config['save_model']) and (epoch+1) % 10 == 0:
            file_name = 'model_{}_{}_val_acc_{:0.4f}_train_acc_{:0.4f}_epoch_{}.pth'.format(
                config['exp_name'], config['task'], val_acc, train_acc, epoch+1)
            torch.save({
                'model_state_dict': model.state_dict()
            }, './weights/{}/{}'.format(config['task'], file_name))

    # 🧪 Final test evaluation
    print("\nEvaluating on test set...")
    test_loss, test_acc = _evaluate_model(
        model, test_loader, val_criterion, epoch, num_epochs, writer, current_lr=0, log_every=0
    )

    print("=" * 50)
    print("FINAL TEST PERFORMANCE:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    print("=" * 50)

    writer.add_scalar('Test/Avg Loss', test_loss, num_epochs)
    writer.add_scalar('Test/Final Accuracy', test_acc, num_epochs)

    # Wrap up
    t_end_training = time.time()
    print(f'Training took {t_end_training - t_start_training:.2f} seconds')
    writer.flush()
    writer.close()


Train the model

In [3]:
print('Training Configuration')
print(config)

train(config=config)

print('Training Ended...')

Training Configuration
{'max_epoch': 50, 'log_train': 100, 'lr': 1e-05, 'starting_epoch': 0, 'batch_size': 1, 'log_val': 10, 'weight_decay': 0.01, 'patience': 5, 'save_model': 1, 'exp_name': 'test'}
Starting to Train Model...
Loading Train Dataset of ACL task...
Class distribution:
Class 0: 7 samples
Class 1: 3 samples
Class 2: 8 samples
Class 3: 4 samples
Class 4: 1 samples
Class weights for loss are: tensor([0.3859, 0.9003, 0.3376, 0.6752, 2.7010])


NameError: name 'neg' is not defined